In [17]:
import random
import math

In [18]:
# Function to parse Solomon instance from file
def parse_solomon_instance(file_path):
    customers = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        vehicle_info = lines[4].split()
        num_vehicles = int(vehicle_info[0])  # Max number of vehicles
        capacity = int(vehicle_info[1])      # Vehicle capacity
        
        for line in lines[9:]:
            data = line.split()
            if len(data) == 7:
                customer = {
                    'id': int(data[0]),
                    'x': float(data[1]),
                    'y': float(data[2]),
                    'demand': float(data[3]),
                    'ready_time': float(data[4]),
                    'due_date': float(data[5]),
                    'service_time': float(data[6])
                }
                customers.append(customer)
    
    depot = customers[0]  # Customer 0 is the depot
    return customers, depot, capacity, num_vehicles

In [19]:
# Distance calculation
def calculate_distance(c1, c2):
    return math.sqrt((c1['x'] - c2['x'])**2 + (c1['y'] - c2['y'])**2)

In [ ]:
# Decode particle position into routes
def decode_position(position, customers, depot, capacity, max_vehicles):
    customer_indices = list(range(1, len(customers)))  # Exclude depot (0)
    sorted_indices = sorted(customer_indices, key=lambda i: position[i-1], reverse=True) #descending order
    
    routes = []
    current_route = []
    current_capacity = 0
    current_time = 0
    
    for idx in sorted_indices:
        customer = customers[idx]
        if len(routes) >= max_vehicles and current_route:  # If max vehicles reached, stop adding new routes
            break
        if current_capacity + customer['demand'] <= capacity:
            last_node = depot if not current_route else customers[current_route[-1]]
            travel_time = calculate_distance(last_node, customer)
            arrival_time = current_time + travel_time
            
            if arrival_time <= customer['due_date']:
                wait_time = max(0, customer['ready_time'] - arrival_time)
                current_time = max(arrival_time, customer['ready_time']) + customer['service_time']
                current_route.append(idx)
                current_capacity += customer['demand']
            else:
                if current_route:
                    routes.append(current_route)
                current_route = [idx]
                current_capacity = customer['demand']
                current_time = customer['ready_time'] + customer['service_time']
        else:
            if current_route:
                routes.append(current_route)
            current_route = [idx]
            current_capacity = customer['demand']
            current_time = customer['ready_time'] + customer['service_time']
    
    if current_route and len(routes) < max_vehicles:
        routes.append(current_route)
    
    # Calculate total distance
    total_distance = 0
    for route in routes:
        prev = depot
        for idx in route:
            total_distance += calculate_distance(prev, customers[idx])
            prev = customers[idx]
        total_distance += calculate_distance(prev, depot)  # Return to depot
    
    return routes, total_distance

In [21]:
# Fitness function with several restriction
def fitness(routes, total_distance, num_customers, max_vehicles):
    num_vehicles = len(routes)
    served_customers = sum(len(route) for route in routes)
    penalty_unserved = 10000 * (num_customers - served_customers)  # Penalize unserved customers
    penalty_vehicles = 100000 * max(0, num_vehicles - max_vehicles)  # Penalize exceeding max vehicles
    return num_vehicles * 1000 + total_distance + penalty_unserved + penalty_vehicles

In [22]:
# Particle class
class Particle:
    def __init__(self, num_customers):
        self.position_i = [random.uniform(0, 1) for _ in range(num_customers-1)]
        self.velocity_i = [random.uniform(-0.1, 0.1) for _ in range(num_customers-1)]
        self.pos_best_i = self.position_i.copy()
        self.err_best_i = float('inf')
        self.err_i = float('inf')

# PSO algorithm
def pso_vrptw(customers, depot, capacity, num_particles, max_iter, max_vehicles):
    swarm = [Particle(len(customers)) for _ in range(num_particles)]
    global_best_pos = None
    global_best_err = float('inf')
    
    for iteration in range(max_iter):
        for particle in swarm:
            routes, distance = decode_position(particle.position_i, customers, depot, capacity, max_vehicles)
            particle.err_i = fitness(routes, distance, len(customers)-1, max_vehicles)
            
            if particle.err_i < particle.err_best_i:
                particle.pos_best_i = particle.position_i.copy()
                particle.err_best_i = particle.err_i
            
            if particle.err_i < global_best_err:
                global_best_pos = particle.position_i.copy()
                global_best_err = particle.err_i
            
            w, c1, c2 = 0.7, 2.0, 2.0
            for d in range(len(particle.position_i)):
                r1, r2 = random.random(), random.random()
                particle.velocity_i[d] = (w * particle.velocity_i[d] +
                                         c1 * r1 * (particle.pos_best_i[d] - particle.position_i[d]) +
                                         c2 * r2 * (global_best_pos[d] - particle.position_i[d]))
                particle.position_i[d] = max(0, min(1, particle.position_i[d] + particle.velocity_i[d]))
    
    routes, distance = decode_position(global_best_pos, customers, depot, capacity, max_vehicles)
    return routes, distance, len(routes)

In [23]:
text_file = 'Data\\c101.txt'
customers, depot, capacity, max_vehicles = parse_solomon_instance(text_file)

# Run PSO with 100 particles and 1000 iterations
routes, distance, num_vehicles = pso_vrptw(customers, depot, capacity, 100, 1000, max_vehicles)

print(f"Instance: c101")
print(f"Maximum Number of Vehicles: {max_vehicles}")
print(f"Number of Vehicles Used: {num_vehicles}")
print(f"Total Distance: {distance:.2f}")
print(f"Routes:")
for i, route in enumerate(routes, 1):
    print(f"  Vehicle {i}: {route}")

Instance: c101
Maximum Number of Vehicles: 25
Number of Vehicles Used: 29
Total Distance: 2727.10
Routes:
  Vehicle 1: [5, 10, 14, 22]
  Vehicle 2: [23]
  Vehicle 3: [25, 26]
  Vehicle 4: [32, 33, 37, 38, 39]
  Vehicle 5: [43, 44, 45, 50, 52]
  Vehicle 6: [54, 56, 59]
  Vehicle 7: [63, 66, 69, 75]
  Vehicle 8: [76, 77, 79]
  Vehicle 9: [81, 83, 84, 85]
  Vehicle 10: [86, 29, 16]
  Vehicle 11: [41, 74, 47]
  Vehicle 12: [13, 3, 100, 2, 1]
  Vehicle 13: [24, 34]
  Vehicle 14: [6]
  Vehicle 15: [20, 27, 28, 68]
  Vehicle 16: [55, 62, 72]
  Vehicle 17: [90, 42, 35, 11]
  Vehicle 18: [78, 88, 89]
  Vehicle 19: [57, 31, 15]
  Vehicle 20: [95, 71, 60, 91, 21, 49]
  Vehicle 21: [98, 96, 4]
  Vehicle 22: [7, 8, 9, 12]
  Vehicle 23: [17, 18, 19, 30, 36]
  Vehicle 24: [40, 46, 48, 51]
  Vehicle 25: [53, 58, 61, 64]
  Vehicle 26: [65]
  Vehicle 27: [67, 70, 73, 80]
  Vehicle 28: [82]
  Vehicle 29: [87, 92, 93]
